In [ ]:
import tensorflow
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

#TensorFlowがGPUを認識しているか確認
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2022-04-30 21:48:00.531558: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4944938076773874112,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 23403757568
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9854097464762977126
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:02:00.0, compute capability: 8.6"]

2022-04-30 21:48:01.050340: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:0 with 22319 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:02:00.0, compute capability: 8.6


# モデル構造の定義
<img src="images/CRNN_SED_DCASE2017_task3.jpg">


In [2]:
from __future__ import print_function

import numpy as np
import time
import sys
import matplotlib.pyplot as plot

from tensorflow.keras.layers import Bidirectional, TimeDistributed, Conv2D, MaxPooling2D, Input, GRU, Dense, Activation, Dropout, Reshape, Permute, LSTM
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# from keras.layers import Dense, Conv2D, SeparableConv2D, Convolution2D, AveragePooling2D, Bidirectional, TimeDistributed, GRU, Reshape, Permute
# from keras.layers import MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Activation, Dropout, BatchNormalization, Flatten, Input
# from keras.models import Model, Sequential
# from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau

from sklearn.metrics import confusion_matrix
import metrics
import utils
from IPython import embed
import keras.backend as K

from tensorflow.keras.utils import plot_model

import tensorflow.experimental.numpy as tnp

# K.set_image_data_format('channels_first')
plot.switch_backend('agg')
sys.setrecursionlimit(10000)

def load_data(_feat_folder, _mono, _fold=None):
    
    # Load name
#     feat_file_fold = os.path.join(_feat_folder, 'mbe_{}_snr0-30_n10s3_fold{}.npz'.format('mon' if _mono else 'bin', _fold))
    feat_file_fold = os.path.join(_feat_folder, 'mbe123_mon_snr_0-28_step2_fold{}.npz'.format(_fold))
        
    dmp = np.load(feat_file_fold)
    _X_train, _Y_train, _X_test, _Y_test = dmp['arr_0'],  dmp['arr_1'],  dmp['arr_2'],  dmp['arr_3']
    return _X_train, _Y_train, _X_test, _Y_test

mel_filt = 40

def get_model(data_in, data_out, _cnn_nb_filt, _cnn_pool_size, _rnn_nb, _fc_nb):
    
    # input_shape (ch, time, mel)
#     spec_start = Input(shape=(data_in.shape[-2], data_in.shape[-1], data_in.shape[-3])) #(time, mel, ch)
    spec_start = Input(shape=(data_in.shape[-3], data_in.shape[-2], data_in.shape[-1])) #default

    spec_x = spec_start
    for _i, _cnt in enumerate(_cnn_pool_size):
        spec_x = Conv2D(filters=_cnn_nb_filt, kernel_size=(3, 3), padding='same', data_format="channels_last")(spec_x)
#         spec_x = BatchNormalization(axis=1)(spec_x)
        spec_x = BatchNormalization(axis=3)(spec_x)
        spec_x = Activation('relu')(spec_x)
#         spec_x = MaxPooling2D(pool_size=(1, _cnn_pool_size[_i]))(spec_x) #cnn_pool_size [5, 2, 2]
        spec_x = MaxPooling2D(pool_size=(_cnn_pool_size[_i], 1))(spec_x) #cnn_pool_size [5, 2, 2]
        spec_x = Dropout(dropout_rate)(spec_x)
#     spec_x = Permute((2, 1, 3))(spec_x)
    spec_x = Permute((1, 3, 2))(spec_x)
#     spec_x = Reshape((data_in.shape[-2], -1))(spec_x) #[-2]:time
    spec_x = Reshape((data_in.shape[-3], -1))(spec_x) #[-2]:time

    for _r in _rnn_nb:
        spec_x = Bidirectional(
            GRU(_r, activation='tanh', dropout=dropout_rate, return_sequences=True),
            merge_mode='mul')(spec_x)

        # GRUは勾配消失により学習が進まないのでLSTMに変更
#         spec_x = Bidirectional(
#             LSTM(_r, dropout=dropout_rate, recurrent_dropout=dropout_rate, return_sequences=True),
#             merge_mode='mul')(spec_x)
 
    for _f in _fc_nb:
        spec_x = TimeDistributed(Dense(_f))(spec_x)
        spec_x = Dropout(dropout_rate)(spec_x)

    spec_x = TimeDistributed(Dense(data_out.shape[-1]))(spec_x)
    out = Activation('sigmoid', name='strong_out')(spec_x)

    _model = Model(inputs=spec_start, outputs=out)
    _model.compile(optimizer="Adam", loss='binary_crossentropy')
    _model.summary()
    
    return _model


def plot_functions(_nb_epoch, _tr_loss, _val_loss, _f1, _er, extension=''):
    plot.figure()

    plot.subplot(211)
    plot.plot(range(_nb_epoch), _tr_loss, label='train loss')
    plot.plot(range(_nb_epoch), _val_loss, label='val loss')
    plot.legend()
    plot.grid(True)

    plot.subplot(212)
    plot.plot(range(_nb_epoch), _f1, label='f')
    plot.plot(range(_nb_epoch), _er, label='er')
    plot.legend()
    plot.grid(True)

    plot.savefig(__models_dir + __fig_name + extension)
    plot.close()
    print('figure name : {}'.format(__fig_name))


def preprocess_data(_X, _Y, _X_test, _Y_test, _seq_len, _nb_ch):
    # split into sequences
    _X = utils.split_in_seqs(_X, _seq_len)
    _Y = utils.split_in_seqs(_Y, _seq_len)

    _X_test = utils.split_in_seqs(_X_test, _seq_len)
    _Y_test = utils.split_in_seqs(_Y_test, _seq_len)

    _X = utils.split_multi_channels(_X, _nb_ch)
    _X_test = utils.split_multi_channels(_X_test, _nb_ch)
    return _X, _Y, _X_test, _Y_test

In [3]:
#######################################################################################
# MAIN SCRIPT STARTS HERE
#######################################################################################

is_mono = True  # True: mono-channel input, False: binaural input

feat_folder = 'feat/'


nb_ch = 1 if is_mono else 2
batch_size = 480 #560    # 576でOOM # Decrease this if you want to run on smaller GPU's
seq_len = 256       # Frame sequence length. Input to the CRNN.
nb_epoch = 150      # Training epochs
patience = int(0.25 * nb_epoch)  # Patience for early stopping

STEP = 10 #generatorで何倍に増量するか


# Number of frames in 1 second, required to calculate F and ER for 1 sec segments.
# Make sure the nfft and sr are the same as in feature.py
sr = 48000
nfft = 1024
frames_1_sec = int(sr/(nfft/2.0))

# Folder for saving model and training curves
__models_dir = 'models/'
utils.create_folder(__models_dir)

# CRNN model definition
cnn_nb_filt = 128            # CNN filter size
cnn_pool_size = [2, 2, 2, 2]   # Maxpooling across frequency. Length of cnn_pool_size =  number of CNN layers
rnn_nb = [32, 32]           # Number of RNN nodes.  Length of rnn_nb =  number of RNN layers
fc_nb = [32]                # Number of FC nodes.  Length of fc_nb =  number of FC layers
dropout_rate = 0.5          # Dropout after each layer

__fig_name = f'mbf{mel_filt}_cnn-f{cnn_nb_filt}-{cnn_pool_size}_bigru{rnn_nb}_fc{fc_nb}_spec{STEP}_e{nb_epoch}p{patience}_batch{batch_size}_{time.strftime("%Y_%m_%d_%H_%M_%S")}'

print('\n\nUNIQUE ID: {}'.format(__fig_name))
print('TRAINING PARAMETERS: nb_ch: {}, seq_len: {}, batch_size: {}, nb_epoch: {}, frames_1_sec: {}'.format(nb_ch, seq_len, batch_size, nb_epoch, frames_1_sec))
print('MODEL PARAMETERS:\n cnn_nb_filt: {}, cnn_pool_size: {}, rnn_nb: {}, fc_nb: {}, dropout_rate: {}'.format(cnn_nb_filt, cnn_pool_size, rnn_nb, fc_nb, dropout_rate))



UNIQUE ID: mbf40_cnn-f128-[2, 2, 2, 2]_bigru[32, 32]_fc[32]_spec10_e150p37_batch480_2022_04_30_21_48_01
TRAINING PARAMETERS: nb_ch: 1, seq_len: 256, batch_size: 480, nb_epoch: 150, frames_1_sec: 93
MODEL PARAMETERS:
 cnn_nb_filt: 128, cnn_pool_size: [2, 2, 2, 2], rnn_nb: [32, 32], fc_nb: [32], dropout_rate: 0.5


In [4]:
X, Y, X_test, Y_test = load_data(feat_folder, is_mono, 1)
print("load.data_X_test:",X_test.shape)
X, Y, X_test, Y_test = preprocess_data(X, Y, X_test, Y_test, seq_len, nb_ch)

# print(X.shape)
# XX=X.transpose(0,2,3,1)
# print(XX.shape)
# XX_test=X_test.transpose(0,2,3,1)
# print("post transpose", XX_test.shape)
# print("default inputshape", X.shape[-3],X.shape[-2],X.shape[-1])
# print("standard inputshape", X.shape[-1],X.shape[-2],X.shape[-3])
print(Y.shape)
print(X.shape)
print(Y.shape)
print(X_test.shape)
print(Y_test.shape)

load.data_X_test: (506265, 40)
(8490, 256, 7)
(8490, 1, 256, 40)
(8490, 256, 7)
(1977, 1, 256, 40)
(1977, 256, 7)


## SpecAugmentのジェネレータ
* ランダムに周波数方向と時間方向にマスクして水増しすることで汎化性能を高める事を目的とする
* 引用論文: https://arxiv.org/abs/1904.08779

In [5]:
import random

random.seed(42)

# class data generator
class SpecaugmentGenerator():
    def __init__(self, x_train, y_train, batch_size=16, alpha=0.2, shuffle=True):
        self.x_train = x_train
        self.y_train = y_train
        self.batch_size = batch_size
        self.alpha = alpha
        self.shuffle = shuffle
        self.sample_num = len(x_train)

    def __call__(self):
        while True:
            indexes = self.__get_exploration_order()
            itr_num = int(len(indexes) // (self.batch_size * 2))

            for i in range(itr_num):
                batch_ids = indexes[i * self.batch_size * 2:(i + 1) * self.batch_size * 2]
                x, y = self.__data_generation(batch_ids)

                yield x, y

    def __get_exploration_order(self):
        indexes = np.arange(self.sample_num)

        if self.shuffle:
            np.random.shuffle(indexes)

        return indexes

    def __data_generation(self, batch_ids):
#         _, h, w, c = self.x_train.shape
#         _, class_num = self.y_train.shape
        
        x1 = self.x_train[batch_ids[:self.batch_size]]
        y = self.y_train[batch_ids[:self.batch_size]]

        for j, _ in enumerate(x1): # shape(batch, time, freq, ch) = x1.shape: (64, 256, 40, 1)
            # 時間軸のマスク
            k = random.randint(0, 50) # max time mask1 width
            l = random.randint(10, 200) # time mask1 start
#             print("specmask - k,l", k,l)
#             print("x1.shape:", x1.shape)
            x1[j, l:l+k, :, :] = 0
    
#             o = random.randint(0, 50) # max mask2 width
#             p = random.randint(250, 450) # mask2 start
#             x1[j, p:p+o, :, :] = 0
            
            # 周波数軸のマスク
            m = random.randint(1, 8) # max freq mask width
            n = random.randint(5, 31)  # freq mask start
            x1[j, :, n:n+m, :] = 0

        x = x1

        return x, y

In [6]:
avg_er = list()
avg_f1 = list()


for fold in [1, 2, 3, 4, 5]:
    print('\n\n----------------------------------------------')
    print('FOLD: {}'.format(fold))
    print('----------------------------------------------\n')
    
    K.clear_session() #モデルを初期化してメモリーリセット
    
    # Load feature and labels, pre-process it
    X, Y, X_test, Y_test = load_data(feat_folder, is_mono, fold)
    print("load.data_X_test:",X_test.shape)
    X, Y, X_test, Y_test = preprocess_data(X, Y, X_test, Y_test, seq_len, nb_ch)
    X=X.transpose(0,2,3,1) #(time, mel, ch)
    X_test=X_test.transpose(0,2,3,1) #(time, mel, ch)
    print(X.shape)
    print(X_test.shape)
    print(Y.shape)
    print(Y_test.shape)

    # Load model
    # Create a MirroredStrategy.
    # strategy = tensorflow.distribute.MirroredStrategy()
    # with strategy.scope():
    model = get_model(X, Y, cnn_nb_filt, cnn_pool_size, rnn_nb, fc_nb)
#     print("created model!")
#     plot_model(model, show_shapes=True)

    # Training
    best_epoch, pat_cnt, best_er, f1_for_best_er, best_conf_mat = 0, 0, 99999, None, None
    tr_loss, val_loss, f1_overall_1sec_list, er_overall_1sec_list = [0] * nb_epoch, [0] * nb_epoch, [0] * nb_epoch, [0] * nb_epoch
    posterior_thresh = 0.5
    
    print(Y.shape)
    training_generator = SpecaugmentGenerator(X, Y, batch_size=batch_size)()
    for i in range(nb_epoch):
        print('Epoch : {} '.format(i), end='')
        
#         # fitで学習
#         hist = model.fit(
#             X, Y,
#             batch_size=batch_size,
#             validation_data=[X_test, Y_test],
#             epochs=1,
#             verbose=0
#         )


        # fit_generatorで学習
        
        hist = model.fit(
                        x=training_generator,
                        steps_per_epoch=X.shape[0] // batch_size * STEP,
                        validation_data=(X_test, Y_test),
                        epochs=1,
                        # validation_steps=X_valid.shape[0] // BATCH_SIZE,
                        validation_steps=None,
                        verbose=1,
                        shuffle=True)

        val_loss[i] = hist.history.get('val_loss')[-1]
        tr_loss[i] = hist.history.get('loss')[-1]

        # Calculate the predictions on test data, in order to calculate ER and F scores
#         print("X_test_shape:",X_test.shape)
#         print("X_test:",X_test)
        pred = model.predict(X_test)
#         print("pred:",pred)
#         print(pred.shape)
        pred_thresh = pred > posterior_thresh
#         print("pred_thresh:",pred_thresh)
#         print(pred_thresh.shape)
        score_list = metrics.compute_scores(pred_thresh, Y_test, frames_in_1_sec=frames_1_sec)
#         print("score_list:",score_list)

        f1_overall_1sec_list[i] = score_list['f1_overall_1sec']
        er_overall_1sec_list[i] = score_list['er_overall_1sec']
        pat_cnt = pat_cnt + 1

        # Calculate confusion matrix
        test_pred_cnt = np.sum(pred_thresh, 2)
        Y_test_cnt = np.sum(Y_test, 2)
        conf_mat = confusion_matrix(Y_test_cnt.reshape(-1), test_pred_cnt.reshape(-1))
        conf_mat = conf_mat / (utils.eps + np.sum(conf_mat, 1)[:, None].astype('float'))

        if er_overall_1sec_list[i] < best_er:
            best_conf_mat = conf_mat
            best_er = er_overall_1sec_list[i]
            f1_for_best_er = f1_overall_1sec_list[i]
            model.save(os.path.join(__models_dir, '{}_fold{}_model.h5'.format(__fig_name, fold)))
            best_epoch = i
            pat_cnt = 0

        print('Train_loss: {}, Val_loss : {}, F1_overall : {}, loss_overall: {} Best_loss: {}, best_epoch: {}'.format(
                tr_loss[i], val_loss[i], f1_overall_1sec_list[i], er_overall_1sec_list[i], best_er, best_epoch))
        plot_functions(nb_epoch, tr_loss, val_loss, f1_overall_1sec_list, er_overall_1sec_list, '_fold_{}'.format(fold))

        # Early stopping
        if pat_cnt > patience:
            break
            
    avg_er.append(best_er)
    avg_f1.append(f1_for_best_er)
    print('saved model for the best_epoch: {} with best_f1: {} f1_for_best_er: {}'.format(
        best_epoch, best_er, f1_for_best_er))
    print('best_conf_mat: {}'.format(best_conf_mat))
    print('best_conf_mat_diag: {}'.format(np.diag(best_conf_mat)))

print('\n\nMETRICS FOR ALL FOUR FOLDS: avg_er: {}, avg_f1: {}'.format(avg_er, avg_f1))
print('MODEL AVERAGE OVER FOUR FOLDS: avg_er: {}, avg_f1: {}'.format(np.mean(avg_er), np.mean(avg_f1)))



----------------------------------------------
FOLD: 1
----------------------------------------------

load.data_X_test: (506265, 40)
(8490, 256, 40, 1)
(1977, 256, 40, 1)
(8490, 256, 7)
(1977, 256, 7)


2022-04-30 21:48:04.412674: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22319 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:02:00.0, compute capability: 8.6


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 40, 1)]      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 40, 128)      1280      
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 40, 128)      512       
_________________________________________________________________
activation (Activation)      (None, 256, 40, 128)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 40, 128)      0         
_________________________________________________________________
dropout (Dropout)            (None, 128, 40, 128)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 40, 128)      147584

2022-04-30 21:48:05.537141: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-30 21:48:09.982154: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8100
2022-04-30 21:48:12.210611: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


170/170 [==============================] - 75s 396ms/step - loss: 0.4140 - val_loss: 0.2104
Train_loss: 0.41401174664497375, Val_loss : 0.2103882133960724, F1_overall : 0.9565985416452705, loss_overall: 0.08217950806320247 Best_loss: 0.08217950806320247, best_epoch: 0
figure name : mbf40_cnn-f128-[2, 2, 2, 2]_bigru[32, 32]_fc[32]_spec10_e150p37_batch480_2022_04_30_21_48_01
170/170 [==============================] - 67s 395ms/step - loss: 0.2259 - val_loss: 0.1371
Train_loss: 0.22585685551166534, Val_loss : 0.13706877827644348, F1_overall : 0.9706077715044835, loss_overall: 0.05607590812835967 Best_loss: 0.05607590812835967, best_epoch: 1
figure name : mbf40_cnn-f128-[2, 2, 2, 2]_bigru[32, 32]_fc[32]_spec10_e150p37_batch480_2022_04_30_21_48_01
170/170 [==============================] - 67s 396ms/step - loss: 0.1858 - val_loss: 0.0983
Train_loss: 0.18583782017230988, Val_loss : 0.09828468412160873, F1_overall : 0.9813233724653149, loss_overall: 0.03624368789705164 Best_loss: 0.0362436878

In [7]:
from keras.utils import plot_model
plot_model(model, to_file='model_.png')

ImportError: cannot import name 'plot_model' from 'keras.utils' (/home/kkishi/anaconda3/envs/tf26/lib/python3.8/site-packages/keras/utils/__init__.py)